# Flask API Lab

First, make sure you can run the existing app (do this in a terminal window):

```bash
(env) $ cd data
(env) $ export FLASK_APP=flask-examples.app7
(env) $ export FLASK_ENV=development 
(env) $ flask run
```

- Create views for commenting on the blog (use the existing BASIC authorization header)

In [1]:
import requests
session = requests.Session()
session.headers['Content-Type'] = 'application/json'
session.auth = 'rick', 'password'

In [3]:
session.get('http://localhost:5000').json()

{'_links': {'posts': 'http://localhost:5000/post'}}

In [4]:
resp = session.post('http://localhost:5000/post', json={
    'title': 'First post', 
    'body': 'Some content'
})

In [5]:
resp.headers['Location']

'http://localhost:5000/post/0a5c166edc0c410793860f6e68b887f9'

In [6]:
resp = session.get(resp.headers['Location']).json()
resp

{'_links': {'self': 'http://localhost:5000/post/0a5c166edc0c410793860f6e68b887f9'},
 'authorName': 'rick',
 'body': 'Some content',
 'postedDate': '2020-07-29T18:50:35.033431',
 'title': 'First post'}

First, I'll update my util library to have get_post_or_404 and get_comment_or_404:

In [12]:
%%file data/flask-examples/util.py
import flask
from flask import abort

state = {'posts': {}}

def url_for(*args, **kwargs):
    return flask.url_for(*args, _external=True, **kwargs)

def get_post_or_404(post_id):
    post = state['posts'].get(post_id)
    if not post:
        abort(404)
    return post
    
def get_comment_or_404(post_id, comment_id):
    post = get_post_or_404(post_id)
    if comment_id < len(post['comments']):
        comment = post['comments'][comment_id]
    else:
        abort(404)
    return comment
    

Overwriting data/flask-examples/util.py


Now update the posts blueprint to use the new util:

In [13]:
%%file data/flask-examples/bp_posts.py
from datetime import datetime
from uuid import uuid4

import flask
from flask import Blueprint, jsonify, request, abort

from .util import state, url_for, get_post_or_404

mod = Blueprint('posts', __name__)

@mod.route('')
def get_posts():
    post_links = [url_for('.get_post', post_id=post_id) for post_id in state['posts']]
    return jsonify(
        _links={'self': url_for('.get_posts')},
        data=[dict(_links=dict(self=link)) for link in post_links])

@mod.route('', methods=['POST'])
def create_post():
    post_id = uuid4().hex
    post = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json,
        'comments': [],
    }
    state['posts'][post_id] = post
    result = jsonify_post(post_id, post)
    result.headers['Location'] = url_for('.get_post', post_id=post_id)
    return result, 201

@mod.route('<post_id>')
def get_post(post_id):
    post = get_post_or_404(post_id)
    return jsonify_post(post_id, post)

@mod.route('<post_id>', methods=['PUT'])
def update_post(post_id):
    post = get_post_or_404(post_id)
    post.update({
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json
    })
    return jsonify_post(id, post)

@mod.route('<post_id>', methods=['DELETE'])
def delete_post(post_id):
    state['posts'].pop(post_id)
    return '', 204

def jsonify_post(post_id, post, **kwargs):
    return jsonify(
        _links={
            'self': url_for('.get_post', post_id=post_id),
            'comments': url_for('comments.get_comments', post_id=post_id)
        },
        postedDate=post['postedDate'].isoformat(),
        authorName=post['authorName'],
        title=post['title'],
        body=post['body']
    )

Overwriting data/flask-examples/bp_posts.py


Create a blueprint for comments:

In [14]:
%%file data/flask-examples/bp_comments.py
from datetime import datetime

import flask
from flask import Blueprint, jsonify, request, abort

from .util import state, url_for, get_post_or_404, get_comment_or_404

mod = Blueprint('comments', __name__)

@mod.route('')
def get_comments(post_id):
    post = get_post_or_404(post_id)
    comment_links = [
        url_for('.get_comment', post_id=post_id, comment_id=i) 
        for i, comment in enumerate(post['comments'])
    ]
    return jsonify(
        _links={'self': url_for('.get_comments', id=post_id)},
        data=[dict(_links=dict(self=link)) for link in comment_links])

@mod.route('', methods=['POST'])
def create_comment(post_id):
    post = get_post_or_404(post_id)
    comment_id = len(post['comments'])
    comment = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json,
    }
    post['comments'].append(comment)
    result = jsonify_comment(post_id, comment_id, comment)
    result.headers['Location'] = url_for('.get_comment', post_id=post_id, comment_id=comment_id)
    return result, 201

@mod.route('<int:comment_id>')
def get_comment(post_id, comment_id):
    comment = get_comment_or_404(post_id, comment_id)
    return jsonify_comment(post_id, comment_id, comment)
    
@mod.route('<int:comment_id>', methods=['PUT'])
def update_comment(post_id, comment_id):
    comment = get_comment_or_404(post_id, comment_id)
    comment.update({
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json,
    })
    return jsonify_comment(post_id, comment_id, comment)

@mod.route('<int:comment_id>', methods=['DELETE'])
def delete_comment(post_id, comment_id):
    post = get_post_or_404(post_id)
    if len(post[comments]) <= comment_id:
        abort(404)
    del post['comments'][comment_id]
    return '', 204

    
def jsonify_comment(post_id, comment_id, comment):
    return jsonify(
        _links={
            'self': url_for('.get_comment', post_id=post_id, comment_id=comment_id),
            'post': url_for('posts.get_post', post_id=post_id)
        },
        authorName=comment['authorName'],
        body=comment['body'],
    )

Overwriting data/flask-examples/bp_comments.py


Finally, create a new top-level app with my new blueprints:

In [17]:
%%file data/flask-examples/lab_blog.py
from flask import Flask, jsonify, url_for

from . import bp_posts
from . import bp_comments
from .util import url_for

app = Flask(__name__)

app.register_blueprint(bp_posts.mod, url_prefix='/post')
app.register_blueprint(bp_comments.mod, url_prefix='/post/<post_id>/comment')

@app.route('/')
def get_root():
    return jsonify(_links={'posts': url_for('posts.get_posts')})

Overwriting data/flask-examples/lab_blog.py


Note that since I changed the app name, I'll need to update my env vars:

```bash
(env) $ cd data
(env) $ export FLASK_APP=flask-examples.lab_blog
(env) $ export FLASK_ENV=development 
(env) $ flask run
```

In [18]:
session.get('http://localhost:5000').json()

{'_links': {'posts': 'http://localhost:5000/post'}}

In [19]:
resp = session.post('http://localhost:5000/post', json={
    'title': 'First post', 
    'body': 'Some content'
})

In [20]:
resp.headers['Location']

'http://localhost:5000/post/e493fe0cd24847398dcec835255e937d'

In [21]:
resp = session.get(resp.headers['Location']).json()
resp

{'_links': {'comments': 'http://localhost:5000/post/e493fe0cd24847398dcec835255e937d/comment',
  'self': 'http://localhost:5000/post/e493fe0cd24847398dcec835255e937d'},
 'authorName': 'rick',
 'body': 'Some content',
 'postedDate': '2020-07-29T18:56:06.291814',
 'title': 'First post'}

In [22]:
resp = session.post(resp['_links']['comments'], json={'body': 'First comment'})
resp

<Response [201]>

In [23]:
resp.headers['Location']

'http://localhost:5000/post/e493fe0cd24847398dcec835255e937d/comment/0'

In [24]:
session.get(resp.headers['Location']).json()

{'_links': {'post': 'http://localhost:5000/post/e493fe0cd24847398dcec835255e937d',
  'self': 'http://localhost:5000/post/e493fe0cd24847398dcec835255e937d/comment/0'},
 'authorName': 'rick',
 'body': 'First comment'}

- Create a blueprint 'authors' which will allow a user to update data about themselves, in particular `fullName` 
  and `password`. 

In [25]:
%%file data/flask-examples/bp_authors.py

import flask
from flask import Blueprint, jsonify, request, abort

from .util import state, url_for, get_author_or_404, jsonify_author

mod = Blueprint('authors', __name__)

@mod.route('<author_id>')
def get_author(author_id):
    author = get_author_or_404(author_id)
    return jsonify_author(author_id, author)

@mod.route('<author_id>', methods=['PUT'])
def update_author(author_id):
    author = get_author_or_404(author_id)
    author.update({
        **request.json,
    })
    return jsonify_author(author_id, author)

Overwriting data/flask-examples/bp_authors.py


- Update the post and comment resources to return author information

First, I need to add a place to store author data. While I'm at it, I'll put all the `jsonify_` helpers in util:

In [26]:
%%file data/flask-examples/util.py
import flask
from flask import jsonify, abort

state = {
    'posts': {},
    'authors': {
        'rick': {
            'fullName': 'Rick Copeland',
            'password': 'seekrit',
        }
    },
}

def url_for(*args, **kwargs):
    return flask.url_for(*args, _external=True, **kwargs)

def get_author_or_404(author_id):
    author = state['authors'].get(author_id)
    if not author:
        abort(404)
    return author
    
def get_post_or_404(post_id):
    post = state['posts'].get(post_id)
    if not post:
        abort(404)
    return post
    
def get_comment_or_404(post_id, comment_id):
    post = get_post_or_404(post_id)
    comment = post['comments'][comment_id]
    return comment
    
def jsonify_post(post_id, post, **kwargs):
    return jsonify(
        _links={
            'self': url_for('posts.get_post', post_id=post_id),
            'comments': url_for('comments.get_comments', post_id=post_id),
            'author': url_for('authors.get_author', author_id=post['author_id']),
        },
        postedDate=post['postedDate'].isoformat(),
        title=post['title'],
        body=post['body']
    )

def jsonify_comment(post_id, comment_id, comment):
    return jsonify(
        _links={
            'self': url_for('comments.get_comment', post_id=post_id, comment_id=comment_id),
            'post': url_for('posts.get_post', post_id=post_id),
            'author': url_for('authors.get_author', author_id=comment['author_id']),
        },
        body=comment['body'],
    )

def jsonify_author(author_id, author):
    return jsonify(
        _links={'self': url_for('authors.get_author', author_id=author_id)},
        fullName=author['fullName'],
    )


Overwriting data/flask-examples/util.py


In [27]:
%%file data/flask-examples/bp_posts.py
from datetime import datetime
from uuid import uuid4

import flask
from flask import Blueprint, jsonify, request, abort

from .util import state, url_for, get_post_or_404, jsonify_post

mod = Blueprint('posts', __name__)

@mod.route('')
def get_posts():
    post_links = [url_for('.get_post', post_id=post_id) for post_id in state['posts']]
    return jsonify(
        _links={'self': url_for('.get_posts')},
        data=[dict(_links=dict(self=link)) for link in post_links])

@mod.route('', methods=['POST'])
def create_post():
    post_id = uuid4().hex
    post = {
        'postedDate': datetime.utcnow(),
        'author_id': request.authorization.username,
        **request.json,
        'comments': [],
    }
    state['posts'][post_id] = post
    result = jsonify_post(post_id, post)
    result.headers['Location'] = url_for('.get_post', post_id=post_id)
    return result, 201

@mod.route('<post_id>')
def get_post(post_id):
    post = get_post_or_404(post_id)
    return jsonify_post(post_id, post)

@mod.route('<post_id>', methods=['PUT'])
def update_post(post_id):
    post = get_post_or_404(post_id)
    post.update({
        'postedDate': datetime.utcnow(),
        'author_id': request.authorization.username,
        **request.json
    })
    return jsonify_post(id, post)

@mod.route('<post_id>', methods=['DELETE'])
def delete_post(post_id):
    state['posts'].pop(post_id)
    return '', 204


Overwriting data/flask-examples/bp_posts.py


In [28]:
%%file data/flask-examples/bp_comments.py
from datetime import datetime

import flask
from flask import Blueprint, jsonify, request, abort

from .util import state, url_for, get_post_or_404, get_comment_or_404, jsonify_comment

mod = Blueprint('comments', __name__)

@mod.route('')
def get_comments(post_id):
    post = get_post_or_404(post_id)
    comment_links = [
        url_for('.get_comment', post_id=post_id, comment_id=i) 
        for i, comment in enumerate(post['comments'])
    ]
    return jsonify(
        _links={'self': url_for('.get_comments', id=post_id)},
        data=[dict(_links=dict(self=link)) for link in comment_links])

@mod.route('', methods=['POST'])
def create_comment(post_id):
    post = get_post_or_404(post_id)
    comment_id = len(post['comments'])
    comment = {
        'postedDate': datetime.utcnow(),
        'author_id': request.authorization.username,
        **request.json,
    }
    post['comments'].append(comment)
    result = jsonify_comment(post_id, comment_id, comment)
    result.headers['Location'] = url_for('.get_comment', post_id=post_id, comment_id=comment_id)
    return result, 201

@mod.route('<int:comment_id>')
def get_comment(post_id, comment_id):
    comment = get_comment_or_404(post_id, comment_id)
    return jsonify_comment(post_id, comment_id, comment)
    
@mod.route('<int:comment_id>', methods=['PUT'])
def update_comment(post_id, comment_id):
    comment = get_comment_or_404(post_id, comment_id)
    comment.update({
        'postedDate': datetime.utcnow(),
        'author_id': request.authorization.username,
        **request.json,
    })
    return jsonify_comment(post_id, comment_id, comment)

@mod.route('<int:comment_id>', methods=['DELETE'])
def delete_comment(post_id, comment_id):
    post = get_post_or_404(post_id)
    if len(post[comments]) <= comment_id:
        abort(404)
    del post['comments'][comment_id]
    return '', 204


Overwriting data/flask-examples/bp_comments.py


Finally, update the app:

In [31]:
%%file data/flask-examples/lab_blog.py
from flask import Flask, jsonify, url_for

from . import bp_posts
from . import bp_comments
from . import bp_authors
from .util import url_for

app = Flask(__name__)

app.register_blueprint(bp_posts.mod, url_prefix='/post')
app.register_blueprint(bp_comments.mod, url_prefix='/post/<post_id>/comment')
app.register_blueprint(bp_authors.mod, url_prefix='/author')

@app.route('/')
def get_root():
    return jsonify(_links={
        'posts': url_for('posts.get_posts'),
    })

Overwriting data/flask-examples/lab_blog.py


In [32]:
session.get('http://localhost:5000').json()

{'_links': {'posts': 'http://localhost:5000/post'}}

In [33]:
resp = session.post('http://localhost:5000/post', json={
    'title': 'First post', 
    'body': 'Some content'
})

In [34]:
resp.headers['Location']

'http://localhost:5000/post/f01f2dcf1b4d42fa8758b0a0d97289b3'

In [35]:
resp = session.get(resp.headers['Location']).json()
resp

{'_links': {'author': 'http://localhost:5000/author/rick',
  'comments': 'http://localhost:5000/post/f01f2dcf1b4d42fa8758b0a0d97289b3/comment',
  'self': 'http://localhost:5000/post/f01f2dcf1b4d42fa8758b0a0d97289b3'},
 'body': 'Some content',
 'postedDate': '2020-07-29T18:58:34.736426',
 'title': 'First post'}

In [36]:
resp = session.post(resp['_links']['comments'], json={'body': 'First comment'})
resp

<Response [201]>

In [37]:
resp.headers['Location']

'http://localhost:5000/post/f01f2dcf1b4d42fa8758b0a0d97289b3/comment/0'

In [38]:
resp = session.get(resp.headers['Location']).json()
resp

{'_links': {'author': 'http://localhost:5000/author/rick',
  'post': 'http://localhost:5000/post/f01f2dcf1b4d42fa8758b0a0d97289b3',
  'self': 'http://localhost:5000/post/f01f2dcf1b4d42fa8758b0a0d97289b3/comment/0'},
 'body': 'First comment'}

In [39]:
session.get(resp['_links']['author']).json()

{'_links': {'self': 'http://localhost:5000/author/rick'},
 'fullName': 'Rick Copeland'}

In [40]:
session.put(resp['_links']['author'], json={'fullName': 'Rick', 'password': 'secret'}).json()

{'_links': {'self': 'http://localhost:5000/author/rick'}, 'fullName': 'Rick'}

- Update your views to verify that the user's password matches the password passed in the authorization header

In [41]:
%%file data/flask-examples/util.py
import hmac

import flask
from flask import jsonify, request, abort

state = {
    'posts': {},
    'authors': {
        'rick': {
            'fullName': 'Rick Copeland',
            'password': 'seekrit',
        }
    },
}

def verify_password():
    if not request.authorization:
        abort(401)
    author = state['authors'].get(request.authorization.username)
    if author is None:
        abort(403)
    good_password = hmac.compare_digest(
        author['password'], request.authorization.password
    )
    # Please note that the approach below is *insecure* 
    #   -- you should use hashing like bcrypt or scrypt to store passwords, 
    #   and you should use a digest comparison
    # good_password = (author['password'] == request.authorization.password)
    if not good_password:
        abort(403)
    

def url_for(*args, **kwargs):
    return flask.url_for(*args, _external=True, **kwargs)

def get_author_or_404(author_id):
    author = state['authors'].get(author_id)
    if not author:
        abort(404)
    return author
    
def get_post_or_404(post_id):
    post = state['posts'].get(post_id)
    if not post:
        abort(404)
    return post
    
def get_comment_or_404(post_id, comment_id):
    post = get_post_or_404(post_id)
    comment = post['comments'][comment_id]
    return comment
    
def jsonify_post(post_id, post, **kwargs):
    return jsonify(
        _links={
            'self': url_for('posts.get_post', post_id=post_id),
            'comments': url_for('comments.get_comments', post_id=post_id),
            'author': url_for('authors.get_author', author_id=post['author_id']),
        },
        postedDate=post['postedDate'].isoformat(),
        title=post['title'],
        body=post['body']
    )

def jsonify_comment(post_id, comment_id, comment):
    return jsonify(
        _links={
            'self': url_for('comments.get_comment', post_id=post_id, comment_id=comment_id),
            'post': url_for('posts.get_post', post_id=post_id),
            'author': url_for('authors.get_author', author_id=comment['author_id']),
        },
        body=comment['body'],
    )

def jsonify_author(author_id, author):
    return jsonify(
        _links={'self': url_for('authors.get_author', author_id=author_id)},
        fullName=author['fullName'],
    )


Overwriting data/flask-examples/util.py


In [42]:
%%file data/flask-examples/bp_posts.py
from datetime import datetime
from uuid import uuid4

import flask
from flask import Blueprint, jsonify, request, abort

from .util import state, url_for, get_post_or_404, jsonify_post, verify_password

mod = Blueprint('posts', __name__)

@mod.route('')
def get_posts():
    post_links = [url_for('.get_post', post_id=post_id) for post_id in state['posts']]
    return jsonify(
        _links={'self': url_for('.get_posts')},
        data=[dict(_links=dict(self=link)) for link in post_links])

@mod.route('', methods=['POST'])
def create_post():
    verify_password()
    post_id = uuid4().hex
    post = {
        'postedDate': datetime.utcnow(),
        'author_id': request.authorization.username,
        **request.json,
        'comments': [],
    }
    state['posts'][post_id] = post
    result = jsonify_post(post_id, post)
    result.headers['Location'] = url_for('.get_post', post_id=post_id)
    return result, 201

@mod.route('<post_id>')
def get_post(post_id):
    post = get_post_or_404(post_id)
    return jsonify_post(post_id, post)

@mod.route('<post_id>', methods=['PUT'])
def update_post(post_id):
    verify_password()
    post = get_post_or_404(post_id)
    post.update({
        'postedDate': datetime.utcnow(),
        'author_id': request.authorization.username,
        **request.json
    })
    return jsonify_post(id, post)

@mod.route('<post_id>', methods=['DELETE'])
def delete_post(post_id):
    verify_password()
    state['posts'].pop(post_id)
    return '', 204


Overwriting data/flask-examples/bp_posts.py


In [43]:
%%file data/flask-examples/bp_comments.py
from datetime import datetime

import flask
from flask import Blueprint, jsonify, request, abort

from .util import state, url_for, get_post_or_404, get_comment_or_404, jsonify_comment, verify_password

mod = Blueprint('comments', __name__)

@mod.route('')
def get_comments(post_id):
    post = get_post_or_404(post_id)
    comment_links = [
        url_for('.get_comment', post_id=post_id, comment_id=i) 
        for i, comment in enumerate(post['comments'])
    ]
    return jsonify(
        _links={'self': url_for('.get_comments', id=post_id)},
        data=[dict(_links=dict(self=link)) for link in comment_links])

@mod.route('', methods=['POST'])
def create_comment(post_id):
    verify_password()
    post = get_post_or_404(post_id)
    comment_id = len(post['comments'])
    comment = {
        'postedDate': datetime.utcnow(),
        'author_id': request.authorization.username,
        **request.json,
    }
    post['comments'].append(comment)
    result = jsonify_comment(post_id, comment_id, comment)
    result.headers['Location'] = url_for('.get_comment', post_id=post_id, comment_id=comment_id)
    return result, 201

@mod.route('<int:comment_id>')
def get_comment(post_id, comment_id):
    comment = get_comment_or_404(post_id, comment_id)
    return jsonify_comment(post_id, comment_id, comment)
    
@mod.route('<int:comment_id>', methods=['PUT'])
def update_comment(post_id, comment_id):
    verify_password()
    comment = get_comment_or_404(post_id, comment_id)
    comment.update({
        'postedDate': datetime.utcnow(),
        'author_id': request.authorization.username,
        **request.json,
    })
    return jsonify_comment(post_id, comment_id, comment)

@mod.route('<int:comment_id>', methods=['DELETE'])
def delete_comment(post_id, comment_id):
    verify_password()
    post = get_post_or_404(post_id)
    if len(post[comments]) <= comment_id:
        abort(404)
    del post['comments'][comment_id]
    return '', 204


Overwriting data/flask-examples/bp_comments.py


In [44]:
%%file data/flask-examples/bp_authors.py

import flask
from flask import Blueprint, jsonify, request, abort

from .util import state, url_for, get_author_or_404, jsonify_author, verify_password

mod = Blueprint('authors', __name__)

@mod.route('<author_id>')
def get_author(author_id):
    author = get_author_or_404(author_id)
    return jsonify_author(author_id, author)

@mod.route('<author_id>', methods=['PUT'])
def update_author(author_id):
    verify_password()
    if request.authorization.username != author_id:
        abort(403)
    author = get_author_or_404(author_id)
    author.update({
        **request.json,
    })
    return jsonify_author(author_id, author)

Overwriting data/flask-examples/bp_authors.py


In [45]:
session.get('http://localhost:5000').json()

{'_links': {'posts': 'http://localhost:5000/post'}}

In [46]:
session.auth = ('rick', 'seekrit')

In [47]:
resp = session.post('http://localhost:5000/post', json={
    'title': 'First post', 
    'body': 'Some content'
})
resp

<Response [201]>

In [48]:
resp.headers['Location']

'http://localhost:5000/post/1284fc561f774bb7999e847d7dd18784'

In [49]:
resp = session.get(resp.headers['Location']).json()
resp

{'_links': {'author': 'http://localhost:5000/author/rick',
  'comments': 'http://localhost:5000/post/1284fc561f774bb7999e847d7dd18784/comment',
  'self': 'http://localhost:5000/post/1284fc561f774bb7999e847d7dd18784'},
 'body': 'Some content',
 'postedDate': '2020-07-29T19:01:05.127766',
 'title': 'First post'}

In [50]:
resp = session.post(resp['_links']['comments'], json={'body': 'First comment'})
resp

<Response [201]>

In [51]:
resp.headers['Location']

'http://localhost:5000/post/1284fc561f774bb7999e847d7dd18784/comment/0'

In [52]:
resp = session.get(resp.headers['Location']).json()
resp

{'_links': {'author': 'http://localhost:5000/author/rick',
  'post': 'http://localhost:5000/post/1284fc561f774bb7999e847d7dd18784',
  'self': 'http://localhost:5000/post/1284fc561f774bb7999e847d7dd18784/comment/0'},
 'body': 'First comment'}

In [53]:
session.get(resp['_links']['author']).json()

{'_links': {'self': 'http://localhost:5000/author/rick'},
 'fullName': 'Rick Copeland'}

In [54]:
session.put(resp['_links']['author'], json={'fullName': 'Rick', 'password': 'secret'}).json()

{'_links': {'self': 'http://localhost:5000/author/rick'}, 'fullName': 'Rick'}

Expect a 403 because we changed the password:

In [55]:
resp = session.post('http://localhost:5000/post', json={
    'title': 'First post', 
    'body': 'Some content'
})
resp

<Response [403]>

In [56]:
resp = requests.post('http://localhost:5000/post', json={
    'title': 'First post', 
    'body': 'Some content'
})
resp

<Response [401]>